In [2]:
# 데이터

In [3]:
!pip install wget

In [4]:
import wget

In [5]:
wget.download('https://raw.githubusercontent.com/e9t/nsmc/master/ratings_train.txt')

'ratings_train (2).txt'

In [119]:
import pandas as pd
df_rating = pd.read_csv('ratings_train.txt', sep='\t')
df_rating[:10]

,id,document,label
0,9976970,아 더빙.. 진짜 짜증나네요 목소리,0
1,3819312,흠...포스터보고 초딩영화줄....오버연기조차 가볍지 않구나,1
2,10265843,너무재밓었다그래서보는것을추천한다,0
3,9045019,교도소 이야기구먼 ..솔직히 재미는 없다..평점 조정,0
4,6483659,사이몬페그의 익살스런 연기가 돋보였던 영화!스파이더맨에서 늙어보이기만 했던 커스틴 ...,1
5,5403919,막 걸음마 뗀 3세부터 초등학교 1학년생인 8살용영화.ㅋㅋㅋ...별반개도 아까움.,0
6,7797314,원작의 긴장감을 제대로 살려내지못했다.,0
7,9443947,별 반개도 아깝다 욕나온다 이응경 길용우 연기생활이몇년인지..정말 발로해도 그것보단...,0
8,7156791,액션이 없는데도 재미 있는 몇안되는 영화,1
9,5912145,왜케 평점이 낮은건데? 꽤 볼만한데.. 헐리우드식 화려함에만 너무 길들여져 있나?,1


In [124]:
df_rating.isnull().query('document==True')

,id,document,label


In [123]:
df_rating.dropna(inplace=True)

In [48]:
!pip install python-mecab-ko

In [59]:
from mecab import MeCab
mecab = MeCab()

In [60]:
mecab.nouns('막 걸음마 뗀 3세부터 초등학교 1학년생인 8살용영화.ㅋㅋㅋ...별반개도 아까움.'), mecab.pos('막 걸음마 뗀 3세부터 초등학교 1학년생인 8살용영화.ㅋㅋㅋ...별반개도 아까움.')

(['걸음마', '세', '초등', '학교', '학년', '살', '영화', '개'],
 [('막', 'MAJ'),
  ('걸음마', 'NNG'),
  ('뗀', 'VV+ETM'),
  ('3', 'SN'),
  ('세', 'NNBC'),
  ('부터', 'JX'),
  ('초등', 'NNG'),
  ('학교', 'NNG'),
  ('1', 'SN'),
  ('학년', 'NNG'),
  ('생', 'XSN'),
  ('인', 'VCP+ETM'),
  ('8', 'SN'),
  ('살', 'NNBC'),
  ('용', 'XSN'),
  ('영화', 'NNG'),
  ('.', 'SF'),
  ('ㅋㅋㅋ', 'IC'),
  ('.', 'SF'),
  ('..', 'SY'),
  ('별반', 'MAG'),
  ('개', 'NNG'),
  ('도', 'JX'),
  ('아까움', 'VA+ETN'),
  ('.', 'SF')])

In [206]:
def extract(series) :
    morphs_nouns_list = []
    for sentence in series:
        morphs_nouns = ''
        for morph, tag in mecab.pos(sentence) : 
            if tag in ['NNG', 'NNP', 'NNB', 'NNBC', 'NP', 'NR', 'VA'] :
                morphs_nouns += (morph+' ')            
        morphs_nouns_list.append(morphs_nouns)
        pass
    return morphs_nouns_list
morphs_nouns_list = extract(df_rating['document'])


In [207]:
df_rating['document_extract'] = morphs_nouns_list
df_rating

,id,document,label,document_extract
0,9976970,아 더빙.. 진짜 짜증나네요 목소리,0,짜증 나 목소리
1,3819312,흠...포스터보고 초딩영화줄....오버연기조차 가볍지 않구나,1,포스터 초딩 영화 줄 오버 연기 가볍
2,10265843,너무재밓었다그래서보는것을추천한다,0,
3,9045019,교도소 이야기구먼 ..솔직히 재미는 없다..평점 조정,0,교도소 이야기 재미 없 평점 조정
4,6483659,사이몬페그의 익살스런 연기가 돋보였던 영화!스파이더맨에서 늙어보이기만 했던 커스틴 ...,1,사이몬페그 익살 연기 영화 스파이더맨 커스틴 던스트
...,...,...,...,...
149995,6222902,인간이 문제지.. 소는 뭔죄인가..,0,인간 문제 소 죄
149996,8549745,평점이 너무 낮아서...,1,평점 낮
149997,9311800,이게 뭐요? 한국인은 거들먹거리고 필리핀 혼혈은 착하다?,0,한국인 필리핀 혼혈 착하
149998,2376369,청춘 영화의 최고봉.방황과 우울했던 날들의 자화상,1,청춘 영화 최고봉 방황 우울 날 자화상


In [186]:
from sklearn.feature_extraction.text import TfidfVectorizer
tfidfVectorizer = TfidfVectorizer(tokenizer=None, ngram_range=(1,2))

In [211]:
tfidfVectorizer.fit(df_rating['document_extract'])

TfidfVectorizer(ngram_range=(1, 2))

In [212]:
features = df_rating['document_extract']
target = df_rating['label']

In [213]:
tokenized_features = tfidfVectorizer.transform(features)

In [214]:
from sklearn.ensemble import RandomForestClassifier
model = RandomForestClassifier(n_jobs=-1)

In [215]:
from sklearn.model_selection import train_test_split
features_train, features_test, target_train, target_test = train_test_split(tokenized_features, target, test_size=0.3, random_state=55)
features_train.shape, features_test.shape, target_train.shape, target_test.shape

((104996, 310318), (44999, 310318), (104996,), (44999,))

In [216]:
model.fit(features_train,target_train)

RandomForestClassifier(n_jobs=-1)

In [218]:
# make predict function
# def sample_predict(features):
#     tokenized_features = tfidfVectorizer.transform(extract(features))
#     return model.predict(tokenized_features)

In [219]:
sample_predict(['필립 생일 파티 명장면. 인생 영화. 다 좋았다.', '참 재밌게 본 영화. 온 가족이 영화관에 쭉 앉아서 본 영화.', '미션 임파서블 시리즈 액션 최고. 아빠가 추천해 보게 된 영화라 다른 의미로 기억에 남는 영화'])

array([1, 1, 1])

In [220]:
# make upgrade function
# def upgrade(x, y):
#     features = df_rating['document']
#     target = df_rating['label']
#     for i in range(len(x)) : 
#         features = features+[x[i]]
#         target = target+[y[i]]
#     features = extract(features)
#     tokenized_features = tfidfVectorizer.fit_transform(features)
#     model.fit(tokenized_features,target)
#     return model

In [221]:
x = ['필립 생일 파티 명장면. 인생 영화. 다 좋았다.', '참 재밌게 본 영화. 온 가족이 영화관에 쭉 앉아서 본 영화.', '미션 임파서블 시리즈 액션 최고. 아빠가 추천해 보게 된 영화라 다른 의미로 기억에 남는 영화']
y = [1,1,1]

In [222]:
from sklearn.metrics import classification_report

In [224]:
target_predict = model.predict(features_test)
print(classification_report(target_test, target_predict))

              precision    recall  f1-score   support

           0       0.71      0.82      0.76     22563
           1       0.79      0.66      0.72     22436

    accuracy                           0.74     44999
   macro avg       0.75      0.74      0.74     44999
weighted avg       0.75      0.74      0.74     44999

